In [1]:
from netgen.occ import *

import sys
sys.path.append("../build/rigid_body")
from rigid_body import *
import rigid_body.bla as bla

import pythreejs as p3

from IPython.display import display
import ipywidgets as widgets

In [2]:
# set up OCC CAD model
box = Box(Pnt(-0.5,-0.5,-0.5), Pnt(0.5,0.5,0.5))
# important: move the center of mass into the origin
box = box.Move((-box.center[0], -box.center[1], -box.center[2]))

In [3]:
# from ngsolve.webgui import Draw
# Draw(mesh)

In [4]:
def mass_matrix_from_solid(obj):
    "extracts the mass matrix of the TopAbs_ShapeEnum.SOLID obj, using the figures computed by netgen"
    
    # copy the inertia matrix from netgen
    inertia_matrix = bla.Matrix(3,3)
    for i in range(3):
        for j in range(3):
            inertia_matrix[i, j] = obj.inertia[i, j]

    # copy the center of mass from netgen
    center_of_mass = bla.Vector(3)
    for i in range(3): center_of_mass[i] = obj.center[i]

    # rearrange it in C++ to make the mass matrix (the elegant way, using MatrixView)
    mass_matrix = mass_matrix_from_inertia(inertia_matrix, center_of_mass, obj.mass)
    return mass_matrix

mass_matrix = mass_matrix_from_solid(box)
# print(mass_matrix)
#for i in range(18):
#    for j in range(18):
#        print(F"{round(mass_matrix[i, j], 5)}, ", end="")
#    print("\\")
def body_from_solid(obj):
    "extracts the mass matrix of the TopAbs_ShapeEnum.SOLID obj, using the figures computed by netgen"
    
    # copy the inertia matrix from netgen
    inertia_matrix = bla.Matrix(3,3)
    for i in range(3):
        for j in range(3):
            inertia_matrix[i, j] = obj.inertia[i, j]

    # copy the center of mass from netgen
    center_of_mass = bla.Vector(3)
    for i in range(3): center_of_mass[i] = obj.center[i]

    # rearrange it in C++ to make the mass matrix (the elegant way, using MatrixView)
    body = RigidBody()
    for i in range(3) : body.center[i] = obj.center[i]
    body.mass = obj.mass
    body.inertia = inertia_matrix
    body.recalcMassMatrix()
    
    return body

In [5]:
def extract_vertices(obj: TopoDS_Shape):
    "extracts a p3js compatible vertex list from a netgen.occ TopoDS_Shape"
    data = obj._webgui_data()["Bezier_trig_points"]
    #print(*data, sep="\n")
    x = data[0]
    y = data[1]
    z = data[2]
    nx = data[3]
    ny = data[4]
    nz = data[5]

    combined_vertices = []
    for i in range(0, len(x), 4):
        combined_vertices.append(x[i : i+3])
        combined_vertices.append(y[i : i+3])
        combined_vertices.append(z[i : i+3])

    combined_normals = []
    for i in range(0, len(nx), 3):
        combined_normals.append(nx[i : i+3])
        combined_normals.append(ny[i : i+3])
        combined_normals.append(nz[i : i+3])
            
    return combined_vertices, combined_normals

In [6]:
view_width = 1000
view_height = 800

# set up pythreejs 3d object
vertices, normals = extract_vertices(box)
buffergeom = p3.BufferGeometry(attributes = {"position" : p3.BufferAttribute(vertices), "normal" : p3.BufferAttribute(normals)})
# buffergeom.exec_three_obj_method("computeVertexNormals") # gives normals for shading
material = p3.MeshPhongMaterial(color='#ff3333', shininess=150, morphTargets=True, side="DoubleSide")
p3mesh = p3.Mesh(buffergeom, material, position=(0,0,0))

# extra scene contents
camera = p3.PerspectiveCamera( position=[10, 6, 10], aspect=view_width/view_height)
key_light = p3.DirectionalLight(position=[0, 10, 10])
ambient_light = p3.AmbientLight()
grid = p3.GridHelper(500, 500//5, "#2F4F4F", "#2F4F4F")

# set up scene
scene = p3.Scene(children=[p3mesh, camera, key_light, ambient_light, grid])
controller = p3.OrbitControls(controlling=camera)
renderer = p3.Renderer(camera=camera, scene=scene, controls=[controller],
                    width=view_width, height=view_height, antialias=True) # if performance is bad, try removing antalias=True

In [7]:
# set up physics simulation object
r =  body_from_solid(box)

t = Transformation()
t.setTranslation(0.1,0,0)
t.setRotation(0,2,0.009)
t.setRotation(2,0,-0.009)
r.dq=t

# save a restore point for the reset button
r.saveState()

In [8]:
print(r.q)

 Translation: 	0.000000 ,0.000000, , 0.000000
 Rotation: 	1.000000 ,0.000000, , 0.000000
 		0.000000 ,1.000000, , 0.000000
		0.000000 ,0.000000, , 1.000000



In [9]:
# play/reset button
play = widgets.Play(
    value=10,
    min=0,
    max=10000,
    step=1,
    interval=100/30,
    description="Press play",
    disabled=False
)

In [10]:
p3mesh.matrixAutoUpdate = False # make mesh movable

def update():
    "update function, gets called every timestep; quasi main loop"
    r.simulate(5,5)
    p3mesh.matrix=r.q.asTuple()

def observer(state):
    "event handler for clickable buttons"
    # if there is a change in time
    if state["name"] == "value":
        # it might be a reset
        if str(state["new"]) == "0":
            r.reset()
            p3mesh.matrix=r.q.asTuple()
        # or it might be a progress in time
        else:
            update()
    # repeat is used as an alias to reset
    elif state["name"] == "repeat":
        r.reset()
        p3mesh.matrix=r.q.asTuple()

play.observe(observer)


In [11]:
display(widgets.HBox([play, widgets.HTML("<b>click-and-drag to rotate, scroll to zoom, right-click-and-drag to move<b>")]))
display(renderer)

Renderer(camera=PerspectiveCamera(aspect=1.25, position=(10.0, 6.0, 10.0), projectionMatrix=(1.0, 0.0, 0.0, 0.…